In [158]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument(r'--user-data-dir=C:\Users\ruoxinli3\AppData\Local\Google\Chrome\User Data')
driver= webdriver.Chrome(options=options)

with open('stealth.min.js', 'r') as f:
    js = f.read()
driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {'source': js})

profile = webdriver.FirefoxProfile()
profile.set_preference('app.update.auto', False)
profile.set_preference('app.update.enabled', False)

driver = webdriver.Firefox(firefox_profile=profile)
driver.get(url)

time.sleep(3)
driver.get("https://seekingalpha.com/author/daniel-jones")

page_str = driver.page_source
print(page_str)
driver.close()  # 关闭浏览器

C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\59594061.py:25: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  profile = webdriver.FirefoxProfile()
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\59594061.py:29: DeprecationWarning: firefox_profile has been deprecated, please pass in an Options object
  driver = webdriver.Firefox(firefox_profile=profile)


<html prefix="og: https://ogp.me/ns#" data-color-scheme="light" style="--scrollbar-width: 17px; --headerFullHeight: 90px; --headerHeight: 90px;" lang="en"><head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<meta charset="utf-8">
<title data-static-tag-from="prerender">Daniel Jones's Articles | Seeking Alpha</title>
<link rel="preconnect" href="https://static.seekingalpha.com">
<link rel="preconnect" href="https://static1.seekingalpha.com">
<link rel="preconnect" href="https://static2.seekingalpha.com">
<link rel="preconnect" href="https://static3.seekingalpha.com">
<link rel="preconnect" href="https://www.googletagmanager.com" crossorigin="anonymous">
<link rel="preconnect" href="https://www.google-analytics.com" crossorigin="anonymous">
<meta name="viewport" content="width=device-width,initial-scale=1,viewport-fit=cover">
<meta content="ie=edge" http-equiv="X-UA-Compatible">
<meta name="application-name" content="Seeking Alpha">
<meta name="HandheldFriendly" co

In [59]:
bs

<!DOCTYPE html>

<html lang="en" prefix="og: https://ogp.me/ns#">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta charset="utf-8"/>
<title data-static-tag-from="prerender">Daniel Jones's Articles | Seeking Alpha</title>
<link href="https://static.seekingalpha.com" rel="preconnect"/>
<link href="https://static1.seekingalpha.com" rel="preconnect"/>
<link href="https://static2.seekingalpha.com" rel="preconnect"/>
<link href="https://static3.seekingalpha.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://www.googletagmanager.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://www.google-analytics.com" rel="preconnect"/>
<meta content="width=device-width,initial-scale=1,viewport-fit=cover" name="viewport"/>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<meta content="Seeking Alpha" name="application-name"/>
<meta content="True" name="HandheldFriendly"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta

In [7]:
import json
import pandas as pd
import numpy as np
import math
import requests

In [3]:
def get_article(article_id):
    url = "https://seeking-alpha.p.rapidapi.com/articles/get-details"

    querystring = {"id":article_id}

    headers = {
        "X-RapidAPI-Key": "78f4b42ff6msh99876ddace67edap1bd499jsn98bcfab8028f",
        "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    return response.text

def get_summary_n_date(response):
    json_object = json.loads(response)
    summary = ""
    
    for s in json_object['data']['attributes']['summary']:
        summary = summary + s
    
    return summary, json_object['data']['attributes']['publishOn']

def get_tickers(response):
    json_object = json.loads(response)
    
    tickers = []
    for key in json_object['data']['relationships']['primaryTickers']:
        for t in json_object['data']['relationships']['primaryTickers'][key]:
            tickers.append (int(t['id']))
    
    tickers_str = []
    for key in json_object['data']['attributes']['themes']:
        
        if json_object['data']['attributes']['themes'][key]['id'] in tickers:
#             print(1)
            tickers_str.append(key.upper())       
    
    return tickers_str
    

In [207]:
raw_df = pd.read_csv('../data/seeking_alpha.csv', header= None)
sa_df =  pd.DataFrame(columns = ['author', 'date', 'ticker','summary'])

for idx,row in raw_df.iterrows():
    articles=[]
    author = row [0]
    
    for i in range(1,43):
        if not math.isnan(row[i]):
            articles.append(str(int(row[i])))
    
    for a in articles:
        response = get_article(a)
        summary, date = get_summary_n_date(response)
        tickers = get_tickers(response)
        for t in tickers:
            sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
    
    print(idx)  

C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykerne

C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykerne

0


C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykerne

1


...

5


C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykerne

C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykerne

6


C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)
C:\Users\ruoxinli3\AppData\Local\Temp\ipykernel_18632\3385929038.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_df = sa_df.append({'author' : author, 'date' : date, 'ticker' :t, 'summary': summary}, ignore_index = True)


In [208]:
sa_df

,author,date,ticker,summary
0,Daniel Jones,2023-03-13T10:45:44-04:00,DJI,"Markets have been volatile, initially starting..."
1,Daniel Jones,2023-03-13T10:45:44-04:00,NDX,"Markets have been volatile, initially starting..."
2,Daniel Jones,2023-03-13T10:45:44-04:00,SP500,"Markets have been volatile, initially starting..."
3,Daniel Jones,2023-03-13T10:45:44-04:00,SPX,"Markets have been volatile, initially starting..."
4,Daniel Jones,2023-03-11T04:53:53-05:00,BMI,"Badger Meter continues to grow, and most of it..."
...,...,...,...,...
229,Hoya Capital,2023-02-27T08:10:00-05:00,MMM,3M is now trading at a very low valuation and ...
230,Hoya Capital,2023-02-27T07:30:00-05:00,ENB,"Enbridge's utilization levels remain high, wit..."
231,Hoya Capital,2023-02-27T07:30:00-05:00,ENB:CA,"Enbridge's utilization levels remain high, wit..."
232,Hoya Capital,2023-02-26T08:00:00-05:00,QRMI,The Global X S&P 500 Risk Managed Income ETF i...


In [210]:
get_tickers(get_article(4586987))

['DJI', 'NDX', 'SP500', 'SPX']

In [212]:
sa_df.to_csv('../data/seeking_alpha_cleaned.csv',index=False)